In [9]:
# DATA PREPROCESSING

#Import necessary packages.
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

#Import necessary data
data2018 = pd.read_csv(r'./Data/FIFAWorldCup2018.csv')
# data2014 = pd.read_csv(r'../Data/FIFAWorldCup2014.csv')
# data2010 = pd.read_csv(r'../Data/FIFAWorldCup2010.csv')
#data2006 = pd.read_csv(r'../Data/FIFAWorldCup2006.csv')

#Stats about data sets
print("Dimensions of FIFA WC 2018: {}rows X {}columns".format(data2018.shape[0],data2018.shape[1]))
# print("Dimensions of FIFA WC 2014: {}rows X {}columns".format(data2014.shape[0],data2014.shape[1]))
# print("Dimensions of FIFA WC 2010: {}rows X {}columns".format(data2010.shape[0],data2010.shape[1]))
# print("Dimensions of FIFA WC 2006: {}rows X {}columns".format(data2006.shape[0],data2006.shape[1]))


Dimensions of FIFA WC 2018: 64rows X 64columns


In [10]:
# DATA INTEGRATION
# dataToConcat = [data2006,data2010,data2014,data2018]
# trainingData = pd.concat([])


In [11]:
# DATA REDUCTION
namesOfColumnsToBeDropped = ["date_GMT","status","attendance","referee","stadium_name", "team_a_xg", "team_b_xg", "btts_percentage_pre_match", 
"over_15_percentage_pre_match", "over_25_percentage_pre_match","over_35_percentage_pre_match", "over_45_percentage_pre_match",
"over_15_HT_FHG_percentage_pre_match", "over_05_HT_FHG_percentage_pre_match", "over_15_2HG_percentage_pre_match", "over_05_2HG_percentage_pre_match","timestamp","Game Week","home_team_goal_timings","away_team_goal_timings",'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg','odds_ft_home_team_win',
 'odds_ft_draw' ,'odds_ft_away_team_win' ,'odds_ft_over15', 'odds_ft_over25',
 'odds_ft_over35', 'odds_ft_over45', 'odds_btts_yes', 'odds_btts_no','total_goal_count', 'total_goals_at_half_time', 'home_team_goal_count_half_time', 'away_team_goal_count_half_time','home_team_first_half_cards', 'home_team_second_half_cards', 'away_team_first_half_cards', 'away_team_second_half_cards','average_goals_per_match_pre_match', 'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match']

dataFrame = data2018.drop(namesOfColumnsToBeDropped,axis=1)

print("Dimensions of FIFA WC 2018 after dropping attributes: {}rows X {}columns".format(dataFrame.shape[0],dataFrame.shape[1]))

# Uncomment next line to display data after dropping unnecessary columns
#display(dataFrame)


Dimensions of FIFA WC 2018 after dropping attributes: 64rows X 20columns


In [21]:
# Feature Engineering

# One-Hot Encoding
labelencoder = LabelEncoder()
dataFrame["home_team_name"] = labelencoder.fit_transform(dataFrame["home_team_name"])
dataFrame["away_team_name"] = labelencoder.fit_transform(dataFrame["away_team_name"])

# Uncomment next line to display data after One-Hot Encoding
#display(dataFrame)

# Adding fields "home_team_result" and "winner_encoded"
winnerEncoded = []
homeTeamResult = []
for index,match in dataFrame.iterrows():
    # when home team wins
    if(match["home_team_goal_count"]>match["away_team_goal_count"]):
        winnerEncoded.append(match["home_team_name"])
        homeTeamResult.append("W")

    # when away team wins
    if(match["home_team_goal_count"]<match["away_team_goal_count"]):
        winnerEncoded.append(match["away_team_name"])
        homeTeamResult.append("L")

    # draws and tie breaker
    if(match["home_team_goal_count"]==match["away_team_goal_count"]):
        if((match["home_team_goal_count"]/match["home_team_shots"])>=(match["away_team_goal_count"]/match["away_team_shots"])):
            winnerEncoded.append(match["home_team_name"])
            homeTeamResult.append("W")
              
        if((match["home_team_goal_count"]/match["home_team_shots"])<(match["away_team_goal_count"]/match["away_team_shots"])):
            winnerEncoded.append(match["away_team_name"])
            homeTeamResult.append("L")

dataFrame["winner_encoded"] = winnerEncoded
dataFrame["home_team_result"] = homeTeamResult

# Uncomment the next line to display dataframe after adding fields "home_team_result" and "winner_encoded"
#display(dataFrame)


In [ ]:
# Writeback to csv data file